In [28]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

import pyLDAvis
import pyLDAvis.gensim_models

In [36]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words



In [38]:
filepath = 'C:/Users/gaoh7/Documents/GitHub/case-outcome-predictor/data/rawish_data/facts.json'
print("Preprocessing text")
data = []
with open(filepath, 'r') as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df = df[['casename', 'facts']]    
df['processed_facts'] = df['facts'].apply(preprocess_text)
df.drop(columns=['facts'], inplace=True)
print(df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
print(lda_model.show_topics())
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

Preprocessing text
0       [fact, appellant, widow, one, tan, geok, tee, ...
1       [fact, surrounding, circumstance, including, a...
2       [background, first, appellant, french, company...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, would, suffice, exh, p...
                              ...                        
8515    [fact, first, accused, low, sze, song, low, ye...
8516    [fact, giving, opinion, representation, would,...
8517    [fact, party, plaintiff, mr, karan, bagga, lit...
8518                                                   []
8519    [fact, party, plaintiff, towa, company, incorp...
Name: processed_facts, Length: 8520, dtype: object
Finished preprocessing text
Performing topic modelling
[(0, '0.021*"accused" + 0.011*"v" + 0.011*"p" + 0.008*"pp" + 0.008*"victim" + 0.008*"statement" + 0.007*"para" + 0.007*"version" + 0.007*"charge" + 0.007*"appellant"'), (1, '0.008*"plaintiff" + 0.007*"defendant" + 0.006*"would" + 0.006*"

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.185513  0.036485       1        1  21.576439
3     -0.046585 -0.006415       2        1  15.982203
4     -0.070215  0.002031       3        1  14.283173
7      0.002466  0.044120       4        1  12.220059
6     -0.067113  0.092317       5        1   7.413948
9      0.012467 -0.061221       6        1   6.313406
2      0.021052 -0.046220       7        1   6.295127
8     -0.033950 -0.032042       8        1   5.881499
1      0.006175 -0.029681       9        1   5.767084
5     -0.009810  0.000626      10        1   4.267062, topic_info=           Term          Freq         Total Category  logprob  loglift
1316  plaintiff  57146.000000  57146.000000  Default  30.0000  30.0000
1992  defendant  56207.000000  56207.000000  Default  29.0000  29.0000
845     accused  26659.000000  26659.000000  Default  28.0000  28.0000
898          mr  52292.000000  52292.000000  Default  27.0000  27.0000
0     appellant  18638.000000  18638.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1992  defendant   1011.320548  56207.834410  Topic10  -5.4739  -0.8636
721           v    871.799637  39358.594181  Topic10  -5.6223  -0.6557
142         day    680.616321  10192.924350  Topic10  -5.8699   0.4478
8          fact    736.815153  17977.015019  Topic10  -5.7905  -0.0403
190       first    688.808033  26325.216547  Topic10  -5.8579  -0.4891

[883 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
6252      1  0.353136      ab
6252      2  0.201649      ab
6252      3  0.155105      ab
6252      4  0.008610      ab
6252      5  0.004492      ab
...     ...       ...     ...
2392      9  0.810960   zoned
2392     10  0.076506   zoned
7899      2  0.039118  zoning
7899      8  0.078236  zoning
7899      9  0.821476  zoning

[4370 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 8, 7, 10, 3, 9, 2, 6])